# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [35]:
# # ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
# mount='/content/gdrive'
# drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

# try:
#   from google.colab import drive
#   IN_COLAB=True
# except:
#   IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [36]:
# # Switch to the directory on the Google Drive that you want to use
# import os
# if IN_COLAB:
#   print("We're running Colab")

#   if IN_COLAB:
#     # Mount the Google Drive at mount
#     print("Colab: mounting Google drive on ", mount)

#     drive.mount(mount)

#     # Create drive_root if it doesn't exist
#     create_drive_root = True
#     if create_drive_root:
#       print("\nColab: making sure ", drive_root, " exists.")
#       os.makedirs(drive_root, exist_ok=True)

#     # Change to the directory
#     print("\nColab: Changing directory to ", drive_root)
#     %cd $drive_root
# # Verify we're in the correct working directory
# %pwd
# print("Archivos en el directorio: ")
# print(os.listdir())

---
### 1.4. Instalar librerías necesarias

In [37]:
# if IN_COLAB:
#   %pip install gym==0.17.3
#   %pip install git+https://github.com/Kojoley/atari-py.git
#   %pip install keras-rl2==1.0.5
#   %pip install tensorflow==2.8
# else:
#   %pip install gym==0.17.3
#   %pip install git+https://github.com/Kojoley/atari-py.git
#   %pip install pyglet==1.5.0
#   %pip install h5py==3.1.0
#   %pip install Pillow==9.5.0
#   %pip install keras-rl2==1.0.5
#   %pip install Keras==2.2.4
#   %pip install tensorflow==2.5.3
#   %pip install torch==2.0.1
#   %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [1]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, Activation, Flatten, Convolution2D, Permute, Input,
                                     SeparableConv2D, Lambda, Subtract, Add, Conv2D, 
                                     DepthwiseConv2D, BatchNormalization, ReLU, InputLayer)
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

import os
import tensorflow as tf
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K 
import time
from tensorflow.keras.callbacks import Callback
from rl.callbacks import FileLogger


#### Configuración base

In [2]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [3]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [4]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


1. Implementación de la red neuronal

# Hiperparámetros

In [31]:
# Hiperparámetros

LEARNING_RATE = 0.0005

GAMMA = 0.99
TARGET_MODEL_UPDATE = 10000
DELTA_CLIP = 1.0
TRAIN_INTERVAL = 24

##### Modelo 4

In [32]:
def create_mobile_like_dqn(input_shape, num_actions):
    x_in = Input(shape=(input_shape[0], input_shape[1], input_shape[2]))
    x = Permute((2, 3, 1))(x_in)  # (84,84,frames)
    # Primer conv estándar ligero
    x = Conv2D(16, (3, 3), strides=2, padding='same')(x)  # ~(42x42x16)
    x = BatchNormalization()(x); x = ReLU()(x)
    # Bloque depthwise-pointwise
    def dw_pw(block_input, filters, strides):
        y = DepthwiseConv2D((3,3), strides=strides, padding='same')(block_input) # aplica una convolución 3x3 a cada canal por separado
        y = BatchNormalization()(y); y = ReLU()(y)
        y = Conv2D(filters, (1,1), padding='same')(y) # combina los canales resultantes, aprendiendo cómo mezclarlos (pointwise)
        y = BatchNormalization()(y); y = ReLU()(y)
        return y
    x = dw_pw(x, 32, strides=2)   # ~(21x21x32)
    x = dw_pw(x, 32, strides=2)   # ~(11x11x32)
    x = dw_pw(x, 64, strides=1)   # ~(11x11x64)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(num_actions, activation='linear', dtype='float32')(x)
    return Model(inputs=x_in, outputs=x)

# Prueba rápida
model = create_mobile_like_dqn((4, 84, 84), num_actions=6)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 4, 84, 84)]       0         
_________________________________________________________________
permute_7 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 42, 42, 16)        592       
_________________________________________________________________
batch_normalization_49 (Batc (None, 42, 42, 16)        64        
_________________________________________________________________
re_lu_49 (ReLU)              (None, 42, 42, 16)        0         
_________________________________________________________________
depthwise_conv2d_21 (Depthwi (None, 21, 21, 16)        160       
_________________________________________________________________
batch_normalization_50 (Batc (None, 21, 21, 16)        64 

In [33]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
num_actions = nb_actions

model_like_dqn = create_mobile_like_dqn(input_shape, num_actions)


2. Implementación de la solución DQN

In [26]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1.0, value_min=0.1, value_test=0.05, nb_steps=1000000)


Callbacks

In [27]:
# Definimos un Callbak para poder guardar los mejores pesos cada 20 iteraciones 
class SaveBestWeights(Callback):
    def __init__(self, filepath, interval=20, overwrite=True):
        super(SaveBestWeights, self).__init__()
        self.filepath = filepath
        self.interval = interval
        self.best_reward = -float('inf')
        self.overwrite = overwrite
    
    def on_episode_end(self, episode, logs={}):
        if episode % self.interval == 0:
            current_reward = logs.get('episode_reward')
            if current_reward > self.best_reward:
                self.best_reward = current_reward
                if self.overwrite or not os.path.exists(self.filepath):
                    self.model.save_weights(self.filepath, overwrite=True)
                    print(f'\n ##### Pesos guardados en {self.filepath} recompensa: {self.best_reward}')
                else:
                    print(f'\n ##### Warning: No se sobrescribió {self.filepath}, ya existe.')


# Definimos un Callbak para poder guardar los ultimos pesos cada 10 iteraciones, sin importar si mejoró el resultado del modelo 
class SaveLastWeights(Callback):
    def __init__(self, filepath, interval=10):
        super(SaveLastWeights, self).__init__()
        self.filepath = filepath
        self.interval = interval
    
    def on_episode_end(self, episode, logs={}):
        if episode % self.interval == 0:
            self.model.save_weights(self.filepath, overwrite=True)
            print(f'\n ##### Pesos guardados en {self.filepath} al final del episodio {episode}')

Agentes

In [34]:
dqn_like_dqn = DQNAgent(
    model=model_like_dqn,
    nb_actions=num_actions,
    policy=policy,
    memory=memory,
    processor=processor,
    nb_steps_warmup = 50000, # indica cuántos pasos (interacciones con el entorno) el agente debe ejecutar antes de comenzar a entrenar el modelo
    gamma=GAMMA,
    target_model_update = TARGET_MODEL_UPDATE,
    train_interval = TRAIN_INTERVAL,
    delta_clip = DELTA_CLIP
)

optimizer = Adam(learning_rate = LEARNING_RATE)
dqn_like_dqn.compile(optimizer=optimizer, metrics=['mae'])

#### Prueba con numero de iteraciones = 100.000

In [ ]:
path = 'POC_train_interval_10_1M'

# Define subdirectorios
videos_path = os.path.join(path, 'videos')
modelos_path = os.path.join(path, 'modelos')

# Crear directorios si no existen
os.makedirs(videos_path, exist_ok=True)
os.makedirs(modelos_path, exist_ok=True)

log_filename = os.path.join(modelos_path, f'dqn_{env_name}_log.json')

filepath_bw = os.path.join(modelos_path, 'dqn_best_weights.h5')
save_best_weights = SaveBestWeights(filepath=filepath_bw, interval=50, overwrite=True)

filepath_lw = os.path.join(modelos_path, 'dqn_last_weights.h5')
save_last_weights = SaveLastWeights(filepath=filepath_lw, interval=50)

print("########--------------  modelo 4  ------------------###########")

# start_time = time.time()

# train_scores = dqn_like_dqn.fit(env, nb_steps=100000, visualize=False, verbose=1, log_interval=25000, 
#                        callbacks=[save_best_weights, save_last_weights, FileLogger(log_filename, interval=100)])

# end_time = time.time()
# training_duration = end_time - start_time

# print(f"Entrenamiento completado en {training_duration:.2f} segundos ({training_duration/60:.2f} minutos)")

weights_filename = './POCS_iters_100000/POC4/modelos/dqn_last_weights.h5'
dqn_like_dqn.load_weights(weights_filename)
test_scores = dqn_like_dqn.test(env, nb_episodes=10, visualize=False)

episode_rewards = test_scores.history['episode_reward']
print(int(np.min(episode_rewards)), "~", int(np.max(episode_rewards)), "Mean", np.mean(episode_rewards))

########--------------  modelo 4  ------------------###########
Training for 100000 steps ...
Interval 1 (0 steps performed)
  641/25000 [..............................] - ETA: 2:52 - reward: 0.0094
 ##### Pesos guardados en POC_train_interval_10_1M\modelos\dqn_best_weights.h5 recompensa: 6.0

 ##### Pesos guardados en POC_train_interval_10_1M\modelos\dqn_last_weights.h5 al final del episodio 0
25000/25000 [==============================] - 191s 8ms/step - reward: 0.0119
38 episodes - episode_reward: 7.605 [3.000, 21.000] - ale.lives: 2.137

Interval 2 (25000 steps performed)
 9489/25000 [==========>...................] - ETA: 1:54 - reward: 0.0130
 ##### Pesos guardados en POC_train_interval_10_1M\modelos\dqn_best_weights.h5 recompensa: 9.0

 ##### Pesos guardados en POC_train_interval_10_1M\modelos\dqn_last_weights.h5 al final del episodio 50
25000/25000 [==============================] - 180s 7ms/step - reward: 0.0128
35 episodes - episode_reward: 9.371 [2.000, 23.000] - ale.lives: 

Resumen de lo obtenido para número de iteraciones = 100.000 


| Model ID | Nombre del Modelo | Recompensas Totales | Recompensa Media | Duración prueba |
|----------|--------------------|----------------------|-------------------|-------------------|
| Model 1  | create_dqn_model         | 11 ~ 28                | 18.6               | 14 m 40 seg 
| Model 2  | create_separable_light_dqn       | 12 ~ 26                | 18.3               | 17 m 58 seg
| Model 3  | create_dueling_light_dqn | 18 ~ 18     | 18               | 17 m
| Model 4  | create_mobile_like_dqn | 18 ~ 20 | 19.6 | 26 m 11 seg

In [35]:
weights_filename = './POCS_iters_100000/POC4/modelos/dqn_last_weights.h5'
dqn_like_dqn.load_weights(weights_filename)
test_scores = dqn_like_dqn.test(env, nb_episodes=10, visualize=False)

episode_rewards = test_scores.history['episode_reward']
print(int(np.min(episode_rewards)), "~", int(np.max(episode_rewards)), "Mean", np.mean(episode_rewards))

Testing for 10 episodes ...


d:\anaconda3\envs\gym4\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Episode 1: reward: 20.000, steps: 714
Episode 2: reward: 20.000, steps: 730
Episode 3: reward: 20.000, steps: 727
Episode 4: reward: 20.000, steps: 729
Episode 5: reward: 20.000, steps: 721
Episode 6: reward: 18.000, steps: 707
Episode 7: reward: 20.000, steps: 718
Episode 8: reward: 20.000, steps: 716
Episode 9: reward: 18.000, steps: 728
Episode 10: reward: 20.000, steps: 724
18 ~ 20 Mean 19.6


In [37]:
def evaluar_pesos(dqn_agent, env, weights_path, nombre_pesos, nb_tests=10, nb_episodes=10):
    dqn_agent.load_weights(weights_path)
    best_mean_reward = -float('inf')
    best_result = None

    for i in range(nb_tests):
        print(f"\n--- Evaluación #{i+1} para {nombre_pesos} ---")
        test_scores = dqn_agent.test(env, nb_episodes=nb_episodes, visualize=False)

        episode_rewards = test_scores.history['episode_reward']
        min_reward = int(np.min(episode_rewards))
        max_reward = int(np.max(episode_rewards))
        mean_reward = np.mean(episode_rewards)

        print(f"Recompensas: {min_reward} ~ {max_reward} | Media: {mean_reward:.2f}")

        if mean_reward > best_mean_reward:
            best_mean_reward = mean_reward
            best_result = {
                "index": i + 1,
                "min": min_reward,
                "max": max_reward,
                "mean": mean_reward
            }
    return best_result


In [38]:
# Ejecutar evaluación para last_weights
weights_last = './POCS_iters_100000/POC4/modelos/dqn_last_weights.h5'
best_last = evaluar_pesos(dqn_like_dqn, env, weights_last, 'last_weights')

print("\n====== Resultados finales ======")
print(f"Mejor resultado para last_weights: Evaluación #{best_last['index']}: {best_last['min']} ~ {best_last['max']} | Media: {best_last['mean']:.2f}")


--- Evaluación #1 para last_weights ---
Testing for 10 episodes ...
Episode 1: reward: 18.000, steps: 721
Episode 2: reward: 18.000, steps: 715
Episode 3: reward: 20.000, steps: 738
Episode 4: reward: 20.000, steps: 732
Episode 5: reward: 18.000, steps: 710
Episode 6: reward: 20.000, steps: 724
Episode 7: reward: 20.000, steps: 729
Episode 8: reward: 18.000, steps: 715
Episode 9: reward: 20.000, steps: 720
Episode 10: reward: 18.000, steps: 719
Recompensas: 18 ~ 20 | Media: 19.00

--- Evaluación #2 para last_weights ---
Testing for 10 episodes ...
Episode 1: reward: 20.000, steps: 727
Episode 2: reward: 20.000, steps: 723
Episode 3: reward: 20.000, steps: 739
Episode 4: reward: 20.000, steps: 723
Episode 5: reward: 20.000, steps: 722
Episode 6: reward: 20.000, steps: 722
Episode 7: reward: 20.000, steps: 737
Episode 8: reward: 18.000, steps: 715
Episode 9: reward: 16.000, steps: 1121
Episode 10: reward: 20.000, steps: 718
Recompensas: 16 ~ 20 | Media: 19.40

--- Evaluación #3 para las

# Mejor resultado para last_weights: Evaluación #3: 18 ~ 27 | Media: 20.30

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---